In [ ]:
from music_generator.synthesizer.oscillators import SineOscillator
from music_generator.basic.signalproc import SamplingInfo
import numpy as np
from multiprocessing import Pool
from keras.layers import Dense, Input, Lambda
from keras.models import Model
import keras.backend as K
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
from music_generator.analysis.play import play_array

In [ ]:
sampling_info = SamplingInfo(44100)

In [ ]:
osc = SineOscillator(sampling_info)

In [ ]:
n_timesteps = 1024
time = n_timesteps / 44100
n_samples = 400000

In [ ]:
def generate_wave_packet(time, amp, freq, phase, location, sigma):
    tone = np.sin(np.arange(n_timesteps) * freq / 44100 * 2 * np.pi + phase)  # osc.generate(1, time, freq, phase)
    envelope = norm.pdf(np.arange(n_timesteps), loc=location, scale=sigma)
    envelope = envelope / (np.max(envelope) * amp)
    return tone * envelope 

In [ ]:
def generate_sample(ii):
    n_freqs = np.random.randint(1, 16)
    amps = np.random.uniform(0.2, 1, size=n_freqs)
    freqs = np.random.uniform(0, sampling_info.nyquist / 16, size=n_freqs)
    locs = np.random.uniform(0, n_timesteps, size=n_freqs)
    sigmas = np.random.uniform(10, n_timesteps * 5, size=n_freqs)    
    phases1 = np.random.uniform(0, 2*np.pi, size=n_freqs)
    phases2 = np.random.uniform(0, 2*np.pi, size=n_freqs)
    x = np.array([generate_wave_packet(time, amp, freq, phase, location, sigma) 
                  for amp, freq, phase, location, sigma in zip(amps, freqs, phases1, locs, sigmas)]).sum(axis=0)
    y = np.array([generate_wave_packet(time, amp, freq, phase, location, sigma) 
                  for amp, freq, phase, location, sigma in zip(amps, freqs, phases2, locs, sigmas)]).sum(axis=0)
    return np.array([x, y]).T.reshape(-1, 2)

In [ ]:
with Pool(8) as pool:
    x_train = np.array(pool.map(generate_sample, range(n_samples)))

In [ ]:
index = np.random.randint(len(x_train))
play_array(x_train[index, :, 0])

In [ ]:
# plt.figure(figsize=[16, 8])
# plt.plot(x_train[index, :, 0])
# plt.plot(x_train[index, :, 1])
# plt.show()

# # index = 0

# index += 1
# play_array(np.repeat(x_train[index, :, 0].reshape(1, -1), 1, axis=0).reshape(-1))
# play_array(np.repeat(x_train[index, :, 1].reshape(1, -1), 1, axis=0).reshape(-1))

In [ ]:
np.mgrid[0:4, 0:5][0].reshape(-1.)

In [ ]:
x_train[index, :].shape

In [ ]:
inp = Input(shape=(x_train[0].shape[0], 2))

inp0 = Lambda(lambda x: x[:, :, 0])(inp)
inp1 = Lambda(lambda x: x[:, :, 1])(inp)

inp_transform0 = Dense(512, use_bias=True)
inp_transform1 = Dense(512, use_bias=True)

freq_space0 = inp_transform0(inp0)
freq_space1 = inp_transform1(inp1)

reco_layer = Dense(x_train[0].shape[0])
reco = reco_layer(freq_space0)

def delta(args):
    freq_space0, freq_space1 = args
    return freq_space0 - freq_space1

delta_layer = Lambda(delta, output_shape=(512,))([freq_space0, freq_space1])

def my_loss(y_true, y_pred):
    reco_true = y_true
    pred_delta_layer, pred_output = delta_layer, y_pred
    mse_hidden = K.mean(K.square(pred_delta_layer))
    mse_reco = K.mean(K.square(pred_output - reco_true))
    return mse_hidden + mse_reco

model = Model(inp, outputs=[delta_layer, reco])
model.summary()
model.compile('Adam', 'mse')

In [ ]:
inp_ae = Input(shape=(x_train[0].shape[0],))
transformed_ae = inp_transform0(inp_ae)
reconstructed_ae = reco_layer(transformed_ae)
ae_model = Model(inp_ae, reconstructed_ae)

In [ ]:
model.fit(x_train, [np.zeros(shape=(len(x_train), 512)), x_train[:, :, 0]], epochs=15, batch_size=1024)

$$h_i = \sum_j W_{ij} x_j + b_i$$

$$x_j = \sum_i \tilde{W}_{ij}(h_i - b_i)$$

In [ ]:
model.save("freq_loss")

In [ ]:
x_inp = osc.generate(1, time, 880, 0).reshape(1, -1)
x_inp.reshape(-1)
y = ae_model.predict(x_inp)
plt.figure(figsize=[16,8])
plt.plot(y.reshape(-1), alpha=0.4)
plt.plot(x_inp.reshape(-1), alpha=0.4)

## Pseudo-inverse

In [ ]:
# activation = np.dot(W[0].T, x_inp) + W[1]
# W_inv = np.linalg.pinv(W[0].T)
# x = np.dot(W_inv, (activation - W[1]))
# plt.figure(figsize=[16,8])
# plt.plot(x, alpha=0.4)
# plt.plot(x_inp, alpha=0.4)